In [1]:
import matplotlib
matplotlib.use('Agg')
from datascience import Table
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
plt.style.use('fivethirtyeight')

from datascience.predicates import are

In [2]:
costs_1 = Table.read_table('costs/outputs-dijkstra-degree-greedy.csv')
costs_2 = Table.read_table('costs/outputs-dijsktra-greedy.csv')

In [3]:
costs_1

output_dir,output_filename,travel_cost,conquer_cost,cost
outputs-dijkstra-degree-greedy,539.out,12004,2816,14820
outputs-dijkstra-degree-greedy,511.out,1.04802e+06,3.50012e+09,3.50117e+09
outputs-dijkstra-degree-greedy,277.out,570,781,1351
outputs-dijkstra-degree-greedy,263.out,1490,424,1914
outputs-dijkstra-degree-greedy,505.out,44,4,48
outputs-dijkstra-degree-greedy,288.out,738,369,1107
outputs-dijkstra-degree-greedy,129.out,8624,3122,11746
outputs-dijkstra-degree-greedy,101.out,324,1,325
outputs-dijkstra-degree-greedy,667.out,1.099e+11,6.19965e+10,1.71897e+11
outputs-dijkstra-degree-greedy,673.out,0.0257,12902,12902


In [4]:
costs_2

output_dir,output_filename,travel_cost,conquer_cost,cost
outputs-dijsktra-greedy,0.out,66,41,107
outputs-dijsktra-greedy,1.out,194,52,246
outputs-dijsktra-greedy,10.out,305.62,351.85,657.47
outputs-dijsktra-greedy,100.out,403864,27704,431568
outputs-dijsktra-greedy,101.out,324,1,325
outputs-dijsktra-greedy,105.out,2.41584e+07,1.48975e+07,3.90559e+07
outputs-dijsktra-greedy,106.out,4.46618e+07,2.97175e+07,7.43793e+07
outputs-dijsktra-greedy,107.out,1.23333e+07,6.44751e+06,1.87808e+07
outputs-dijsktra-greedy,108.out,13308.1,3376.05,16684.2
outputs-dijsktra-greedy,109.out,10799.8,1880.66,12680.5


In [10]:
d = costs_1.join('output_filename', costs_2)
d.where("cost", are.above, "cost_2")
# d

output_filename,output_dir,travel_cost,conquer_cost,cost,output_dir_2,travel_cost_2,conquer_cost_2,cost_2
117.out,outputs-dijkstra-degree-greedy,375,602,977,outputs-dijsktra-greedy,376,520,896
118.out,outputs-dijkstra-degree-greedy,775,1206,1981,outputs-dijsktra-greedy,776,1040,1816
119.out,outputs-dijkstra-degree-greedy,176,300,476,outputs-dijsktra-greedy,176,260,436
121.out,outputs-dijkstra-degree-greedy,63248,1845,65093,outputs-dijsktra-greedy,63300,1171,64471
122.out,outputs-dijkstra-degree-greedy,786,516,1302,outputs-dijsktra-greedy,810,452,1262
126.out,outputs-dijkstra-degree-greedy,274,720,994,outputs-dijsktra-greedy,306,600,906
145.out,outputs-dijkstra-degree-greedy,992.26,1048,2040.26,outputs-dijsktra-greedy,1141.12,837,1978.12
146.out,outputs-dijkstra-degree-greedy,133,91,224,outputs-dijsktra-greedy,149,65,214
157.out,outputs-dijkstra-degree-greedy,32,189064,189096,outputs-dijsktra-greedy,30,173498,173528
159.out,outputs-dijkstra-degree-greedy,760,11724,12484,outputs-dijsktra-greedy,874,10176,11050


In [6]:
appended = costs_1.copy().append(costs_2).sort('output_filename')
appended

output_dir,output_filename,travel_cost,conquer_cost,cost
outputs-dijkstra-degree-greedy,0.out,28,40,68
outputs-dijsktra-greedy,0.out,66,41,107
outputs-dijkstra-degree-greedy,1.out,194,52,246
outputs-dijsktra-greedy,1.out,194,52,246
outputs-dijkstra-degree-greedy,10.out,250.8,281.31,532.11
outputs-dijsktra-greedy,10.out,305.62,351.85,657.47
outputs-dijkstra-degree-greedy,100.out,403864,27704,431568
outputs-dijsktra-greedy,100.out,403864,27704,431568
outputs-dijkstra-degree-greedy,101.out,324,1,325
outputs-dijsktra-greedy,101.out,324,1,325


In [7]:
with_min = appended.group('output_filename', min).select(['output_filename', 'cost min'])
with_min

output_filename,cost min
0.out,68
1.out,246
10.out,532.11
100.out,431568
101.out,325
105.out,3.58966e+07
106.out,7.11494e+07
107.out,1.59049e+07
108.out,12658.1
109.out,11609.3


In [8]:
min_output_selected = appended.join('output_filename', with_min).where('cost', are.equal_to, 'cost min').drop('cost min')
min_output_selected

output_filename,output_dir,travel_cost,conquer_cost,cost
0.out,outputs-dijkstra-degree-greedy,28,40,68
1.out,outputs-dijkstra-degree-greedy,194,52,246
1.out,outputs-dijsktra-greedy,194,52,246
10.out,outputs-dijkstra-degree-greedy,250.8,281.31,532.11
100.out,outputs-dijkstra-degree-greedy,403864,27704,431568
100.out,outputs-dijsktra-greedy,403864,27704,431568
101.out,outputs-dijkstra-degree-greedy,324,1,325
101.out,outputs-dijsktra-greedy,324,1,325
105.out,outputs-dijkstra-degree-greedy,1.78909e+07,1.80057e+07,3.58966e+07
106.out,outputs-dijkstra-degree-greedy,3.59132e+07,3.52362e+07,7.11494e+07


In [67]:
# To implement: select files from min_out_selected and copy into a new folder
